In [39]:
#import libraries
import pandas as pd
import requests
import json
import datetime as dt

In [40]:
#Read excel file with actual man hour per Part Number, Serial Number and Work Order
df = pd.read_excel(r'C:\Users\paulo\OneDrive\SHOP - HH Total Prog x HH Total Apontado.xlsx',sheet_name='HH PROG X HH APONTADO')

In [41]:
#link from BCB to get US Dolar per day
url  = r"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='01-01-2020'&@dataFinalCotacao='12-31-2023'&$top=9997&$format=json&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao"
response = requests.get(url)

In [42]:
#Create a dataframe with US Dolar value
dolar = pd.DataFrame(response.json()['value'])

In [43]:
#format 'dataHoracotacao from 'dolar' to date
dolar['date'] = pd.to_datetime(dolar['dataHoraCotacao']).dt.date

In [44]:
#Format 'CREATED_DATE' from 'df' to date
df['date'] = pd.to_datetime(df['CREATED_DATE'], format='%d/%m/%Y').dt.date

In [45]:
#Merge dataframes
df = df.merge(dolar, how = 'left', on = ['date'])

In [46]:
#Remove NaN values
df = df[df['H_H_TOTAL_REGISTRADO'].notna()]
df = df[df['cotacaoCompra'].notna()]

In [51]:
#Multiply 'H_H_TOTAL_REGISTRADO' x 100 (valor unitário homem hora) / 'cotacaocompra' dia
df['Valor_USD'] = df['H_H_TOTAL_REGISTRADO'] * 100 / df['cotacaoCompra']

In [52]:
df

,PN,PN_SN,PN_DESCRIPTION,WO,H_H_TOTAL_REGISTRADO,H_H_MEDIO_P_N,SITE,LOCATION,CREATED_DATE,date,cotacaoCompra,cotacaoVenda,dataHoraCotacao,Valor_USD
0,PW127M,ED1089,ENGINE PW100 LARGE SERIES - APPLICABLE FOR AT...,418631,87.82,43.4,SHOP02,PLU,02/01/2020,2020-01-02,4.0207,4.0213,2020-01-02 13:11:10.762,2184.196782
1,170-43903-403,AD14242,AFT TOILET SHROUD ASSY,419034,8.82,16.3,INTERIORES,PLU,06/01/2020,2020-01-06,4.0548,4.0554,2020-01-06 13:03:22.271,217.519976
2,AZL1185-R00,NaN,COCKPIT PROTECTION - LH,419035,0.77,0.7,INTERIORES,PLU,06/01/2020,2020-01-06,4.0548,4.0554,2020-01-06 13:03:22.271,18.989839
3,AZL1184-R00,NaN,COCKPIT PROTECTION - RH,419036,0.63,0.7,INTERIORES,PLU,06/01/2020,2020-01-06,4.0548,4.0554,2020-01-06 13:03:22.271,15.537141
4,PR- AQK,NaN,NaN,419037,102.08,102.1,SHOP03,PLU,06/01/2020,2020-01-06,4.0548,4.0554,2020-01-06 13:03:22.271,2517.510111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16936,C20195162-51C,62051,"MLG WHEEL, TIRE ASSY",507650,0.77,8.0,SHOP,VCP-OFI-RF,24/08/2021,2021-08-24,5.3011,5.3017,2021-08-24 13:10:22.53,14.525287
16937,C20195162-51C,62811,"MLG WHEEL, TIRE ASSY",507651,1.70,8.0,SHOP,VCP-OFI-RF,24/08/2021,2021-08-24,5.3011,5.3017,2021-08-24 13:10:22.53,32.068816
16938,90000581-1WTC,OCT08-2503,"WHEEL ASSY, NLG",507652,2.83,8.1,SHOP,VCP-OFI-RF,24/08/2021,2021-08-24,5.3011,5.3017,2021-08-24 13:10:22.53,53.385146
16939,196-62014-405,E0035,NLG WHEEL AND TIRE ASSY,507653,1.90,6.8,SHOP,VCP-OFI-RF,24/08/2021,2021-08-24,5.3011,5.3017,2021-08-24 13:10:22.53,35.841618
